In [2]:
import sys
import os
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
import plotly_express as px
from datetime import datetime, timedelta
import plotly.graph_objects as go
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
from sklearn.model_selection import TimeSeriesSplit
from xgboost import XGBRegressor


module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
from src.paths import TRANSFORMED_DATA_DIR, MODELS_DIR
from src.feature import train_test_kfolds


In [4]:
# Read the transformed data , included the topn features and the time features
wind_farms_data_features_target = pd.read_parquet(f'{TRANSFORMED_DATA_DIR}/wind_farm_topn_features.parquet')

In [10]:
# Create train test folds
time_split = TimeSeriesSplit(gap=24, test_size=93, n_splits=10)
target_features_required = list(wind_farms_data_features_target.columns)                                                
train_test_folds= train_test_kfolds(wind_farms_data_features_target, time_split=time_split, features_target=target_features_required)

In [11]:

result = {}
fig = px.line(
    wind_farms_data_features_target,
    x=wind_farms_data_features_target.index,
    y="CF",
    markers=True,
    hover_data=["CF"]
)
for key, value in train_test_folds.items():

    X_train, y_train, X_test, y_test = train_test_folds[key]
    model = XGBRegressor(n_estimators=300, random_state=42, eta=0.1)
    model.fit(X_train,y_train)
    predictions = model.predict(X_test)
    result[key]= (mean_absolute_error(y_test, predictions), mean_absolute_percentage_error(y_test, predictions))

    fig.add_trace(go.Scatter(x=y_test.index, y=predictions, mode='lines', name=str(key), line=dict(color='red')))

print(f"Mean abs percentage error, MAPE:{np.mean([v[1] for v in result.values()])}") # Ignoring mape based on goal and low predicted values

print(f"Mean abs error, MAE:{np.mean([v[0] for v in result.values()])}")

fig.update_layout(title="Predicted vs Actuals")
# fig.add_annotation(dict(text="Max training date"))

fig.update_layout(annotations=[
    dict(
        x=1.05,  # x and y coordinates for positioning, adjust as needed
        y=1.02,
        xref='paper',
        yref='paper',
        text='Max Training date',  # Legend title text
        showarrow=False,
        align='right'
    )
])
fig.show()

c:\Users\gkris\.conda\envs\vistra\lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



Mean abs percentage error, MAPE:0.4833699218998161
Mean abs error, MAE:0.04213730032224074


Hypertuning the XGBoost model 

In [12]:
from sklearn.model_selection import RandomizedSearchCV, TimeSeriesSplit



X = wind_farms_data_features_target.values[:,1:]
y = wind_farms_data_features_target.values[:,0]


# Create a time series cross-validator
tscv = TimeSeriesSplit(gap=24, test_size=93, n_splits=10)


# Parameter grid
param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.3],
    'max_depth': [3, 5, 7],
    'min_child_weight': [1, 3, 5],
    'subsample': [0.7, 0.8, 0.9],
    'colsample_bytree': [0.7, 0.8, 0.9],
    'gamma': [0, 0.1, 0.2]
}

# XGBRegressor
xgb_reg = XGBRegressor()

# Randomized search
random_search = RandomizedSearchCV(xgb_reg, param_grid, n_iter=50, 
                                   scoring='neg_mean_absolute_error', 
                                   cv=tscv, random_state=42)

random_search.fit(X, y)

# Best parameters
print(random_search.best_params_)

{'subsample': 0.8, 'n_estimators': 200, 'min_child_weight': 1, 'max_depth': 3, 'learning_rate': 0.1, 'gamma': 0.1, 'colsample_bytree': 0.9}


There is an improvement in the accuracy of the model after hypertuning  
The fold with January 5 and January 6 have interpolated data as the actuals , this could be leading to larger absolute error than realized

In [15]:
result = {}
params=random_search.best_params_
fig = px.line(
    wind_farms_data_features_target,
    x=wind_farms_data_features_target.index,
    y="CF",
    markers=True,
    hover_data=["CF"]
)
for key, value in train_test_folds.items():

    X_train, y_train, X_test, y_test = train_test_folds[key]
    model = XGBRegressor(**params)
    model.fit(X_train,y_train)
    predictions = model.predict(X_test)
    result[key]= (mean_absolute_error(y_test, predictions), mean_absolute_percentage_error(y_test, predictions))

    fig.add_trace(go.Scatter(x=y_test.index, y=predictions, mode='lines', name=str(key), line=dict(color='red')))

print(f"Mean abs percentage error, MAPE:{np.mean([v[1] for v in result.values()])}") # Ignoring mape based on goal and low predicted values

print(f"Mean abs error, MAE:{np.mean([v[0] for v in result.values()])}")

fig.update_layout(title="Predicted vs Actuals, After tuning MAE: 0.04081")
# fig.add_annotation(dict(text="Max training date"))

fig.update_layout(annotations=[
    dict(
        x=1.05,  # x and y coordinates for positioning, adjust as needed
        y=1.02,
        xref='paper',
        yref='paper',
        text='Max Training date',  # Legend title text
        showarrow=False,
        align='right'
    )
])
fig.show()

c:\Users\gkris\.conda\envs\vistra\lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



Mean abs percentage error, MAPE:0.4753513904814583
Mean abs error, MAE:0.0408119439955283
